In [1]:
import os

spark_version = 'spark-3.2.3'
os.environ['SPARK_VERSION']=spark_version

# Install Spark and Java
!apt-get update
!apt-get install openjdk-11-jdk-headless -qq > /dev/null
!wget -q http://www.apache.org/dist/spark/$SPARK_VERSION/$SPARK_VERSION-bin-hadoop2.7.tgz
!tar xf $SPARK_VERSION-bin-hadoop2.7.tgz
!pip install -q findspark

# Set Environment Variables
import os
os.environ["JAVA_HOME"] = "/usr/lib/jvm/java-11-openjdk-amd64"
os.environ["SPARK_HOME"] = f"/content/{spark_version}-bin-hadoop2.7"

# Start a SparkSession
import findspark
findspark.init()

Hit:1 http://ppa.launchpad.net/c2d4u.team/c2d4u4.0+/ubuntu focal InRelease
Hit:2 https://cloud.r-project.org/bin/linux/ubuntu focal-cran40/ InRelease
Hit:3 http://ppa.launchpad.net/cran/libgit2/ubuntu focal InRelease
Hit:4 http://ppa.launchpad.net/deadsnakes/ppa/ubuntu focal InRelease
Hit:5 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu2004/x86_64  InRelease
Hit:6 http://ppa.launchpad.net/graphics-drivers/ppa/ubuntu focal InRelease
Hit:7 http://ppa.launchpad.net/ubuntugis/ppa/ubuntu focal InRelease
Hit:8 http://archive.ubuntu.com/ubuntu focal InRelease
Get:9 http://security.ubuntu.com/ubuntu focal-security InRelease [114 kB]
Hit:10 http://archive.ubuntu.com/ubuntu focal-updates InRelease
Get:11 http://archive.ubuntu.com/ubuntu focal-backports InRelease [108 kB]
Fetched 222 kB in 1s (201 kB/s)
Reading package lists... Done


In [2]:
!wget https://jdbc.postgresql.org/download/postgresql-42.2.17.jar

--2023-03-29 06:32:06--  https://jdbc.postgresql.org/download/postgresql-42.2.17.jar
Resolving jdbc.postgresql.org (jdbc.postgresql.org)... 72.32.157.228, 2001:4800:3e1:1::228
Connecting to jdbc.postgresql.org (jdbc.postgresql.org)|72.32.157.228|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 1004734 (981K) [application/java-archive]
Saving to: ‘postgresql-42.2.17.jar.2’

postgresql-42.2.17. 100%[===================>] 981.19K  1.59MB/s    in 0.6s    

2023-03-29 06:32:08 (1.59 MB/s) - ‘postgresql-42.2.17.jar.2’ saved [1004734/1004734]



In [3]:
from pyspark.sql import SparkSession
spark = SparkSession.builder.appName("CloudETL").config("spark.driver.extraClassPath","/content/postgresql-42.2.17.jar").getOrCreate()

In [4]:
# Read in data from S3 Buckets
from pyspark import SparkFiles
url ="https://casey-elizabeth-haley-kaleb-bucket.s3.us-east-2.amazonaws.com/emissions_data.csv"
spark.sparkContext.addFile(url)
emissions_data = spark.read.csv(SparkFiles.get("emissions_data.csv"), sep=",", header=True, inferSchema=True)

In [5]:
url ="https://casey-elizabeth-haley-kaleb-bucket.s3.us-east-2.amazonaws.com/population_data.csv"
spark.sparkContext.addFile(url)
population_data = spark.read.csv(SparkFiles.get("population_data.csv"), sep=",", header=True, inferSchema=True)

In [6]:
url ="https://casey-elizabeth-haley-kaleb-bucket.s3.us-east-2.amazonaws.com/death_data.csv"
spark.sparkContext.addFile(url)
death_data = spark.read.csv(SparkFiles.get("death_data.csv"), sep=",", header=True, inferSchema=True)

In [7]:
url ="https://casey-elizabeth-haley-kaleb-bucket.s3.us-east-2.amazonaws.com/emissions_dataframe.csv"
spark.sparkContext.addFile(url)
emissions_dataframe = spark.read.csv(SparkFiles.get("emissions_dataframe.csv"), sep=",", header=True, inferSchema=True)

In [8]:
from getpass import getpass
password = getpass('Enter database password')

mode = "append"
jdbc_url="jdbc:postgresql://group-4-database.c0mbmuajokyj.us-east-2.rds.amazonaws.com:5432/postgres"
config = {"user":"postgres",
          "password": password,
          "driver":"org.postgresql.Driver"}

Enter database password··········


In [9]:
emissions_data.write.jdbc(url=jdbc_url, table='emissions_data', mode=mode, properties=config)

In [10]:
population_data.write.jdbc(url=jdbc_url, table='population_data', mode=mode, properties=config)

AnalysisException: ignored

In [ ]:
death_data.write.jdbc(url=jdbc_url, table='death_data', mode=mode, properties=config)

In [ ]:
emissions_dataframe.write.jdbc(url=jdbc_url, table='emissions_dataframe', mode=mode, properties=config)